In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import astropy.coordinates as coord
import astropy.table as at
import astropy.units as u
from pyia import GaiaData
from scipy.interpolate import interp1d
from scipy.ndimage import gaussian_filter

coord.galactocentric_frame_defaults.set('v4.0');

In [ ]:
run_name = '150pc_MG12-result'
t = at.Table.read(f'../cache/probs-{run_name}.fits')
d = at.Table.read(f'../data/{run_name}.fits.gz')
joined = at.join(d, t, keys='source_id', join_type='left')

g = GaiaData(joined[joined['prob'] > 0.5])
allg = GaiaData(joined)

---

# CMDs

In [ ]:
mg = g.phot_g_mean_mag - g.distmod
bprp = g.phot_bp_mean_mag - g.phot_rp_mean_mag

In [ ]:
iso = at.Table.read('/mnt/home/apricewhelan/data/isochrones/PARSEC/FeH_0.4_iso.fits')
iso = iso[np.isclose(iso['log(age/yr)'], 8.2)]
iso = iso[iso['stage'] < 2]

In [ ]:
iso_interp = interp1d(iso['G'], iso['G_BP']-iso['G_RP'])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))

ax.plot(bprp, mg, marker='o', ls='none', alpha=0.5, mew=0)

ax.plot(iso['G_BP']-iso['G_RP'], iso['G'])

pred_bprp = iso_interp(mg.value)
cmd_mask = (np.abs(bprp.value - pred_bprp) < 0.15) #& (mg < 9*u.mag)
ax.plot(bprp[cmd_mask], mg[cmd_mask], 
        marker='o', ls='none', alpha=0.5, mew=0, color='tab:red')

ax.set_xlim(-0.5, 3.5)
ax.set_ylim(12.5, -4)

ax.set_xlabel('BP-RP')
ax.set_ylabel('$M_G$')

fig.set_facecolor('w')

In [ ]:
c = g.get_skycoord()
gal = c.galactic

allc = allg.get_skycoord()
allgal = allc.galactic

In [ ]:
xyz = gal.cartesian.xyz.to_value(u.pc)

In [ ]:
labels = ['x', 'y', 'z']

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for k, (i, j) in enumerate([(0,1), (0,2), (1,2)]):
    ax = axes[k]

    ax.plot(xyz[i][cmd_mask],
            xyz[j][cmd_mask],
            marker='o', ls='none', mew=0, ms=5, alpha=0.5)

    ax.set_xlabel(f'${labels[i]}$')
    ax.set_ylabel(f'${labels[j]}$')
fig.tight_layout()
fig.set_facecolor('w')

In [ ]:
allvxyz = allgal.velocity.d_xyz.to_value(u.km/u.s)
vxyz = gal.velocity.d_xyz.to_value(u.km/u.s)
vxyz = np.nanmedian(vxyz[:, cmd_mask], axis=1)
vxyz

In [ ]:
# control_v0 = np.array([15, 5, -2.5])
# derp = coord.SkyCoord(coord.CartesianRepresentation([10,10,10]*u.pc).with_differentials(coord.CartesianDifferential(control_v0*u.km/u.s)), 
#                       frame='galactic').icrs.velocity

control_v0 = np.array([15, 5, -2.5])
derp = coord.SkyCoord(coord.CartesianRepresentation([10,10,10]*u.pc).with_differentials(coord.CartesianDifferential(control_v0*u.km/u.s)), 
                      frame='icrs').galactic.velocity
control_v0 = derp.d_xyz.value
control_v0

In [ ]:
for cuddly in [True, False]:
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    for k, (i, j) in enumerate([(0,1), (0,2), (1,2)]):
        ax = axes[k]

        H, xe, ye = np.histogram2d(allvxyz[i],
                                   allvxyz[j],
                                   bins=np.arange(-75, 50+1e-3, 1))
        
#         ax.pcolormesh(xe, ye, H.T, 
#                       norm=mpl.colors.LogNorm(), cmap='Greys');
        if cuddly:
            H = H - gaussian_filter(H, sigma=3)
            ax.pcolormesh(xe, ye, H.T, cmap='Greys', vmin=-20, vmax=20)
        else:
            ax.pcolormesh(xe, ye, H.T, cmap='Greys')
        
        ax.scatter(vxyz[i], vxyz[j],
                   marker='o', zorder=100, linewidth=2, facecolor='none', 
                   alpha=0.75, s=250, edgecolor='tab:red')
        
        ax.scatter(control_v0[i],
                   control_v0[j],
                   marker='o', zorder=100, linewidth=2, facecolor='none', 
                   alpha=0.75, s=150, edgecolor='tab:orange')
            
        ax.set_xlabel(f'$v_{labels[i]}$')
        ax.set_ylabel(f'$v_{labels[j]}$')
    fig.tight_layout()
    fig.set_facecolor('w')

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

fig1d, axes1d = plt.subplots(1, 3, figsize=(15, 5))

not_in = [2, 1, 0]
for k, (i, j) in enumerate([(0,1), (0,2), (1,2)]):
    ax = axes[k]
    
    mask = np.abs(allvxyz[not_in[k]] - vxyz[not_in[k]]) < 2.
    print(mask.sum())
    H, xe, ye = np.histogram2d(allvxyz[i][mask],
                               allvxyz[j][mask],
                               bins=np.arange(-60, 30+1e-3, 0.5))
    H = gaussian_filter(H, sigma=1.5)
    ax.pcolormesh(xe, ye, H.T, cmap='Greys')
    
    circ = mpl.patches.Circle((vxyz[i], vxyz[j]),
                              radius=2, 
                              facecolor='none', edgecolor='tab:red', lw=1.)
    ax.add_patch(circ)
#     ax.scatter(vxyz[i], vxyz[j],
#                marker='o', zorder=100, linewidth=2, facecolor='none', 
#                alpha=0.75, s=250, edgecolor='tab:red')

    ax.set_xlabel(f'$v_{labels[i]}$')
    ax.set_ylabel(f'$v_{labels[j]}$')
    
    # --------------
    # 1D histograms:
    ax = axes1d[k]
    
    circ_mask = circ.get_path().contains_points(np.stack((allvxyz[i], allvxyz[j])).T)
    ax.hist(allvxyz[not_in[k]][circ_mask], bins=np.arange(-60, 30, 2))
    ax.axvline(vxyz[not_in[k]], color='tab:red')
    print(circ_mask.sum())
    
fig.tight_layout()
fig.set_facecolor('w')

fig1d.tight_layout()
fig1d.set_facecolor('w')

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for k in range(3):
    ax = axes[k]
    
    mask = np.abs(allvxyz[not_in[k]] - vxyz[not_in[k]]) < 1.
    print(mask.sum())
    H, xe, ye = np.histogram2d(allvxyz[i][mask],
                               allvxyz[j][mask],
                               bins=(np.arange(0, 80, 2),
                                     np.arange(-60, 30+1e-3, 0.5)))
    H = gaussian_filter(H, sigma=1.5)
    ax.pcolormesh(xe, ye, H.T, cmap='Greys')

    ax.set_xlabel(f'$v_{labels[i]}$')
    ax.set_ylabel(f'$v_{labels[j]}$')
    
fig.tight_layout()
fig.set_facecolor('w')